In [1]:
# 1. Ortam değişkenlerini (API Key) yükle
from dotenv import load_dotenv
import os

# .env dosyasını okur ve sisteme yükler
load_dotenv()

# Kontrol: Key gelmiş mi? (Güvenlik için sadece ilk 5 harfini yazdırıyoruz)
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ API Key başarıyla yüklendi: {api_key[:5]}...")
else:
    print("❌ HATA: API Key bulunamadı! .env dosyanı ve ismini kontrol et.")

# 2. Gerekli kütüphaneleri çağıralım
from langchain_openai import ChatOpenAI
from youtube_transcript_api import YouTubeTranscriptApi

# 3. Modeli Başlat (GPT-3.5-Turbo kullanıyoruz, test için ucuz ve hızlıdır)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 4. Test Videosu: "Me at the zoo" (YouTube'un ilk videosu, kısa olduğu için test etmesi kolay)
video_id = "jNQXAC9IVRw" 

try:
    print("⏳ Altyazı çekiliyor...")
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    
    # Parça parça gelen altyazıyı tek bir metin haline getir
    full_text = " ".join([i['text'] for i in transcript])
    print(f"✅ Altyazı alındı! Toplam karakter: {len(full_text)}")
    
    # 5. Yapay Zekaya Gönder ve Cevabı Al
    print("🤖 Yapay zeka düşünüyor...")
    prompt = f"Aşağıdaki video metnini bana Türkçe olarak 1 cümleyle özetle:\n\n{full_text}"
    response = llm.invoke(prompt)
    
    print("\n" + "="*30)
    print("SONUÇ:")
    print(response.content)
    print("="*30)

except Exception as e:
    print(f"❌ Bir hata oluştu: {e}")

✅ API Key başarıyla yüklendi: sk-pr...
⏳ Altyazı çekiliyor...
❌ Bir hata oluştu: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'


In [2]:
import youtube_transcript_api
print("Kütüphane Dosya Yolu:", youtube_transcript_api.__file__)
print("İçindekiler:", dir(youtube_transcript_api.YouTubeTranscriptApi))


Kütüphane Dosya Yolu: c:\Users\yavuz\AppData\Local\Programs\Python\Python311\Lib\site-packages\youtube_transcript_api\__init__.py
İçindekiler: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fetch', 'list']


In [1]:
# Gerekli kütüphaneleri çağır
from langchain_community.document_loaders import YoutubeLoader
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Şifreleri yükle
load_dotenv()

# Yapay Zeka Modelini Hazırla
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Test Videosu: "Me at the zoo"
video_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw"

try:
    print("⏳ Video işleniyor...")
    
    # LangChain'in kendi yükleyicisi (Daha stabil çalışır)
    loader = YoutubeLoader.from_youtube_url(
        video_url, 
        add_video_info=False,
        language=["tr", "en"] # Önce Türkçe ara, yoksa İngilizce al
    )
    
    # Videoyu yükle ve metne çevir
    docs = loader.load()
    icerik = docs[0].page_content
    
    print(f"✅ Başarılı! Metin uzunluğu: {len(icerik)} karakter.")
    print("-" * 20)
    
    # Yapay Zekaya Özetlet
    print("🤖 Özet çıkarılıyor...")
    ozet_prompt = f"Aşağıdaki video metnini Türkçe olarak, samimi bir dille özetle:\n\n{icerik}"
    cevap = llm.invoke(ozet_prompt)
    
    print("SONUÇ:\n", cevap.content)

except Exception as e:
    print(f"❌ Hata Detayı: {e}")

⏳ Video işleniyor...
✅ Başarılı! Metin uzunluğu: 217 karakter.
--------------------
🤖 Özet çıkarılıyor...
❌ Hata Detayı: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [3]:
# Gerekli kütüphaneler
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import YoutubeLoader
from dotenv import load_dotenv
import os

# 1. Şifreleri yükle
load_dotenv()

# Google Key kontrolü
if not os.getenv("GOOGLE_API_KEY"):
    print("❌ HATA: .env dosyasında GOOGLE_API_KEY bulunamadı!")
else:
    print("✅ Google API Key bulundu.")

# 2. Modeli Başlat (Gemini 1.5 Flash kullanıyoruz - Bedava ve Hızlı)
llm = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0.7)

# 3. Test Videosu (Yine bizim filler videosu)
video_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw"

try:
    print("⏳ Video işleniyor...")
    
    # Videoyu çek
    loader = YoutubeLoader.from_youtube_url(
        video_url, 
        add_video_info=False,
        language=["tr", "en"]
    )
    docs = loader.load()
    icerik = docs[0].page_content
    
    print(f"✅ Video metni alındı! ({len(icerik)} karakter)")
    
    # 4. Yapay Zekaya Gönder
    print("🤖 Gemini düşünüyor...")
    prompt = f"Bu video metnini Türkçe olarak, esprili bir dille özetle:\n\n{icerik}"
    cevap = llm.invoke(prompt)
    
    print("\n" + "="*30)
    print("SONUÇ (GEMINI):")
    print(cevap.content)
    print("="*30)

except Exception as e:
    print(f"❌ Bir hata oluştu: {e}")

✅ Google API Key bulundu.
⏳ Video işleniyor...
✅ Video metni alındı! (217 karakter)
🤖 Gemini düşünüyor...
❌ Bir hata oluştu: Error calling model 'gemini-flash-latest' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. ', 'status': 'RESOURCE_EXHAUSTED'}}


In [1]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Şifreyi al
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("❌ API Key bulunamadı!")
else:
    # Google kütüphanesini ayarla
    genai.configure(api_key=api_key)

    print("🔍 Senin hesabına açık olan modeller listeleniyor...\n")
    
    found_models = []
    try:
        for m in genai.list_models():
            # Sadece metin üretme (chat) yeteneği olanları filtrele
            if 'generateContent' in m.supported_generation_methods:
                print(f"👉 {m.name}")
                found_models.append(m.name)
        
        if not found_models:
            print("\n⚠️ Hiçbir model bulunamadı. API Key veya bölge ayarlarında sorun olabilir.")
            
    except Exception as e:
        print(f"❌ Listeleme Hatası: {e}")

C:\Users\yavuz\AppData\Local\Temp\ipykernel_10856\1410708389.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


🔍 Senin hesabına açık olan modeller listeleniyor...

👉 models/gemini-2.5-flash
👉 models/gemini-2.5-pro
👉 models/gemini-2.0-flash-exp
👉 models/gemini-2.0-flash
👉 models/gemini-2.0-flash-001
👉 models/gemini-2.0-flash-exp-image-generation
👉 models/gemini-2.0-flash-lite-001
👉 models/gemini-2.0-flash-lite
👉 models/gemini-2.0-flash-lite-preview-02-05
👉 models/gemini-2.0-flash-lite-preview
👉 models/gemini-exp-1206
👉 models/gemini-2.5-flash-preview-tts
👉 models/gemini-2.5-pro-preview-tts
👉 models/gemma-3-1b-it
👉 models/gemma-3-4b-it
👉 models/gemma-3-12b-it
👉 models/gemma-3-27b-it
👉 models/gemma-3n-e4b-it
👉 models/gemma-3n-e2b-it
👉 models/gemini-flash-latest
👉 models/gemini-flash-lite-latest
👉 models/gemini-pro-latest
👉 models/gemini-2.5-flash-lite
👉 models/gemini-2.5-flash-image-preview
👉 models/gemini-2.5-flash-image
👉 models/gemini-2.5-flash-preview-09-2025
👉 models/gemini-2.5-flash-lite-preview-09-2025
👉 models/gemini-3-pro-preview
👉 models/gemini-3-flash-preview
👉 models/gemini-3-pro-image